(I've decided to go with JSON format for now)

In [1]:
import json

In [2]:
# get excluded obs, sug
excl_obs = set()
with open('../susan_obs_sug_by_hand/obs_residue_for_getting_excluded_lines.tsv') as f:
    next(f)
    for line in f:
        ln = line.strip().split('\t')
        if len(ln) == 3:
            if '?' in ln[-1]:
#                 print(ln[1])
                pass
            else:
                pass
        else:
            assert len(ln) == 2
            if 'UNCAT' in ln[0]:
                excl_obs.add(ln[1])

In [3]:
with open('../FINAL_OBS_AND_SUG.json') as f:
    data = json.load(f)
all_obs = [
    o.replace(',', '').replace('.', '').replace('/', ' ').replace('word by word', 'word-by-word').strip()
    for ex in data
    for ev in ex['Evaluations']
    for o in ev['Observations']
]
all_obs[:] = (' '.join(x.split()).replace('"', '') for x in all_obs)
# with open('../obs_loaded_from_json.json', 'x') as f:
#     json.dump(all_obs, f, indent=2)
all_sug = [
    s.replace(',', '').replace('.', '').replace('word by word', 'word-by-word').strip().replace('"', '')
    for ex in data
    for ev in ex['Evaluations']
    for s in ev['Suggestions']
]
# with open('../sug_loaded_from_json.json', 'x') as f:
#     json.dump(all_sug, f, indent=2)

In [4]:
with open('../susan_obs_sug_by_hand/OBS_JSON_CLEAN.json') as f:
    obs_to_cats = json.load(f)
new = {}
for x, y in obs_to_cats.items():
    x = x.replace(',', '').replace('.', '').replace('word by word', 'word-by-word').strip()
    x = ' '.join(x.split())
    new[x] = y
obs_to_cats = new

In [5]:
obs_dont_have = set()
for o in all_obs:
    if o in excl_obs:
        continue
        
    if o in obs_to_cats:
        pass

In [6]:
for x in obs_dont_have:
    print(x)

In [7]:
len(obs_dont_have)

0

In [8]:
all_obs = [
    o.replace(',', '').replace('.', '').replace('/', ' ').replace('word by word', 'word-by-word').strip()
    for ex in data
    for ev in ex['Evaluations']
    for o in ev['Observations']
]
lost = 0
for ex in data:
    for ev in ex['Evaluations']:
        observations_to_collections = {}
        for o in ev['Observations']:
            _o_ = o.replace(',', '').replace('.', '').replace('/', ' ').replace('word by word', 'word-by-word').replace('"', '').strip()
            _o_ = ' '.join(_o_.split())
            if _o_ not in obs_to_cats and _o_ not in excl_obs:
                lost += 1
            if _o_ in obs_to_cats:
                categories = sorted(obs_to_cats[_o_])
                if len(categories) != 1:
                    print(categories)
                observations_to_collections[_o_] = categories
        ev['Observations'] = observations_to_collections

['EXPRESSION', 'PHRASING', 'RATE']
['EXPRESSION', 'RATE']
['ACCURACY', 'MONITORING_FOR_MEANING']
['MONITORING_FOR_MEANING', 'MULTISYLLABIC_WORDS']
['FLUENCY', 'RATE']
['RATE', 'SUBSTITUTION_REVERSAL']
['FLUENCY', 'PHRASING', 'RATE']
['FLUENCY', 'RATE']
['OMISSION_INSERTION', 'SUBSTITUTION_REVERSAL']
['MONITORING_FOR_MEANING', 'SUBSTITUTION_REVERSAL']
['OMISSION', 'SUBSTITUTION_REVERSAL']
['EXPRESSION', 'PHRASING', 'RATE']
['ACCURACY', 'MONITORING_FOR_MEANING']
['FLUENCY', 'MONITORING_FOR_MEANING']
['ACCURACY', 'MONITORING_FOR_MEANING']
['PHRASING', 'RATE']
['OMISSION', 'RATE', 'SUBSTITUTION_REVERSAL']
['OMISSION_INSERTION', 'SUBSTITUTION_REVERSAL']
['ACCURACY', 'MONITORING_FOR_MEANING']
['ACCURACY', 'MONITORING_FOR_MEANING']
['PHRASING', 'RATE']


In [9]:
def normalize_string(s):
    s = s.replace(',', '').replace('multi-syllable', 'multisyllabic').replace('.', '').replace('"', '').replace("'", '')
    return ' '.join(s.split())

In [10]:
with open('../susan_obs_sug_by_hand/SUG_JSON_CLEAN.json') as f:
    labeled_sugs = json.load(f)

normalized_labeled_sugs = {}
for s, cats in labeled_sugs.items():
    normalized_labeled_sugs[normalize_string(s)] = cats

In [11]:
# add suggestion categories
not_found = []
for ex in data:
    for ev in ex['Evaluations']:
        sug_to_cats = {}
        for sug in ev['Suggestions']:
            if normalize_string(sug) not in normalized_labeled_sugs:
                not_found.append(normalize_string(sug))
            elif 'UNCATEGORIZED' not in normalized_labeled_sugs[normalize_string(sug)]:
                sug_to_cats[sug] = normalized_labeled_sugs[normalize_string(sug)]
        ev['Suggestions'] = sug_to_cats
# print(len(not_found))
# not_found
# with open('../ALL_OBS_SUGS_AND_CATEGORIES.json', 'x') as f:
#     json.dump(data, f, indent=2)